In [186]:
import pandas as pd
import numpy as np
import re
from kiwipiepy import Kiwi
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import pymysql


In [187]:
con = pymysql.connect(
    host='localhost',
    user='root',
    password='1111',
    db='yorehalre',
    charset='utf8'
)

data = pd.read_sql("SELECT recipeID, recipe_name, summary, ingredient_name, category FROM recipe_info", con)
data_re = pd.read_sql("SELECT category, recipeID, recipe_name, user_id, rating, review_content FROM user_review", con)

con.close()

C:\Users\SSO\AppData\Local\Temp\ipykernel_1692\2245140572.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql("SELECT recipeID, recipe_name, summary, ingredient_name, category FROM recipe_info", con)
C:\Users\SSO\AppData\Local\Temp\ipykernel_1692\2245140572.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_re = pd.read_sql("SELECT category, recipeID, recipe_name, user_id, rating, review_content FROM user_review", con)


In [188]:
# data = pd.read_csv('../data/weaviate_data(임시).csv')
data.head(2)

,recipeID,recipe_name,summary,ingredient_name,category
0,0,우엉양배추볶음밥,찬밥이 기다려지는 우엉양배추볶음밥입니다.,[재료] 찬 밥 적당량 양배추 적당량 우엉 적당량 느타리버섯 조금 다시마 ...,저칼로리 다이어트
1,1,아삭아삭 연근장아찌 만들기,연근은 비타민C와 식이섬유소가 풍부하고 빈혈예방에 탁월하고 코피가 자주나는 아이들에...,[재료] 연근 [양념비율(200ml 계량컵 기준)] 간장 1 물 1 설탕 0...,저칼로리 다이어트


In [189]:
source = []
for ingredient_name in data['ingredient_name']:
    ingredient_name_str = str(ingredient_name)
    if '양념' in ingredient_name_str:
        ingredient_matches = re.findall(r'\[.*?\](.*?)\양념', ingredient_name_str)
        source_per_ingredient = []
        for ingredient_match in ingredient_matches:
            ingredient_cleaned = re.sub(r'[0-9]+[가-힣]+', '', ingredient_match).strip().split()
            for ing in ingredient_cleaned:
                if ing:
                    ingredient_final = re.sub(r'[^가-힣\s]', '', ing).strip()
                    if ingredient_final:
                        source_per_ingredient.extend(ingredient_final.split())
    else:
        ingredient_matches = re.findall(r'\[.*?\](.*)', ingredient_name_str)
        source_per_ingredient = []
        for ingredient_match in ingredient_matches:
            ingredient_cleaned = re.sub(r'[0-9]+[가-힣]+', '', ingredient_match).strip().split()
            for ing in ingredient_cleaned:
                if ing:
                    ingredient_final = re.sub(r'[^가-힣\s]', '', ing).strip()
                    if ingredient_final:
                        source_per_ingredient.extend(ingredient_final.split())
    # if source_per_ingredient:
    source.append(source_per_ingredient)

In [190]:
len(source)

874

In [191]:
sum(1 for item in source if not item)

32

In [192]:
data['main_source'] = source

In [193]:
# data_re = pd.read_csv('../data/user_review.csv')

In [194]:
data_re.head(3)

,category,recipeID,recipe_name,user_id,rating,review_content
0,간 건강,765,카레 해물 파스타,3366,3,참고해서 잘 해먹었습니다~ 우유가 없어서 카레를 그냥 재료들과 볶았는데 나쁘지 않았어요
1,간 건강,768,생생정보 오이미역냉국 황금레시피,1013,5,애들이 맛있다고 잘먹어요..
2,간 건강,768,생생정보 오이미역냉국 황금레시피,1173,5,정말 맛나요~


In [195]:
df = pd.merge(data, data_re, on=['recipeID', 'recipe_name', 'category'], how='outer')

In [196]:
kiwi = Kiwi()
def extract_keywords(text):
    stop_words = ['물', '약간', '적당', '적당량', '조금', '때', '레시피', '요리', '저', '거', '것', '수', '나']
    keywords = []
    text = text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    for title in text:
        titles = []
        result = kiwi.analyze(title)
        for token, tag, _ , _ in result[0][0]:
            if tag[0] in 'NN':
                if token not in stop_words and not re.match(r'.*\W.*', token):
                    titles.append(token.strip())
        keywords.append(titles)
    return keywords

In [197]:
name = extract_keywords(df['recipe_name'].apply(str))
summary = extract_keywords(df['summary'].apply(str))
source = extract_keywords(df['main_source'].apply(str))

In [198]:
review = extract_keywords(df['review_content'].apply(str))


In [199]:
list_all = []

for item_1, item_2, item_3, item_4 in zip(name, summary, df['main_source'], review):
    combined_list = item_1 + item_2 + item_3 + item_4
    list_all.append(combined_list)

In [200]:
df['all'] = list_all

In [201]:
all_keyword = []
for i in df['all']:
    for item in i:
        all_keyword.append(item)
print(len(all_keyword))

86131


In [202]:
unique_keyword = set(all_keyword)
# print(unique_keyword)
print(len(unique_keyword))


4022


In [203]:
'우엉' in unique_keyword

True

In [204]:
total_recipe_count = len(set(data['recipe_name']))
keyword_count_dict = dict.fromkeys(unique_keyword)

for keyword_list in df['all']:
    for keyword in keyword_list:
        if keyword_count_dict[keyword.strip()] == None:
            keyword_count_dict[keyword.strip()] = 1
        else:
            keyword_count_dict[keyword.strip()] += 1

In [205]:
keyword_idf = dict()
for each_keyword in keyword_count_dict:
    keyword_idf[each_keyword] = np.log10(total_recipe_count/keyword_count_dict[each_keyword])

In [206]:
sorted_data_dict = dict(sorted(keyword_idf.items(), key=lambda item: item[1], reverse=True))

In [207]:
all_category = data['category'].explode().tolist()
all_keyword = df['all'].explode().tolist()
unique_keyword = set(all_keyword)
unique_category = set(all_category)

In [208]:
unique_category

{'간 건강',
 '골다공증',
 '노화방지',
 '머슬업 다이어트',
 '변비',
 '스트레스 해소',
 '암 예방',
 '위 건강',
 '장 건강',
 '저칼로리 다이어트',
 '치매예방',
 '피로회복',
 '혈당조절',
 '혈액순환',
 '호흡기 건강'}

## category 별 데이터프레임

In [209]:
category_representation = pd.DataFrame(columns=sorted(unique_category), index=df['recipeID'])
print(category_representation.head(1))

         간 건강 골다공증 노화방지 머슬업 다이어트   변비 스트레스 해소 암 예방 위 건강 장 건강 저칼로리 다이어트 치매예방  \
recipeID                                                                      
0         NaN  NaN  NaN      NaN  NaN     NaN  NaN  NaN  NaN       NaN  NaN   

         피로회복 혈당조절 혈액순환 호흡기 건강  
recipeID                        
0         NaN  NaN  NaN    NaN  


In [210]:
for category in df['category']:
    # 해당하는 카테고리에는 1을 넣고 아니면 0을 넣음
    category_representation[category] = np.where(df['category'] == category, 2, None)

In [211]:
print(category_representation.tail(5))

          간 건강  골다공증  노화방지 머슬업 다이어트    변비 스트레스 해소  암 예방  위 건강  장 건강 저칼로리 다이어트  \
recipeID                                                                        
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
873       None  None  None     None  None    None  None  None  None      None   

         치매예방  피로회복  혈당조절  혈액순환 호흡기 건강  
recipeID                                
872         2  None  None  None   None  
872         2  None  None  None   None  
872         2  None  None  None   None  
872         2  None  None  None   None  
873         2  None  None  None   None  


## keyword 별 데이터프레임

In [212]:
keyword_representation = pd.DataFrame(columns=sorted(unique_keyword), index=df['recipeID'])
print(keyword_representation.head(1))

          2손다  3스픈    ᆢ  ᆢ저는    ㅋ   ㅋ넘   ㅋ존    ㅎ   ㅡ넘    가  ...  흰우유   흰자  \
recipeID                                                    ...             
0         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   

           흰콩  흰후추 흰후추가루    히   히트   힌트   힐링    힘  
recipeID                                           
0         NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  

[1 rows x 4022 columns]


In [213]:
for index in tqdm(range(len(df['recipeID']))):
    keywords = df['all'][index]
    recipeID = df['recipeID'].iloc[index]
    for keyword in keywords:
        keyword_idf_value = keyword_idf.get(keyword.strip(), None) 
        if keyword_idf_value is not None:
            keyword_representation.at[recipeID, keyword] = keyword_idf_value

# keyword_representation.sort_index(inplace=True)

100%|██████████| 4025/4025 [00:06<00:00, 637.58it/s]


In [214]:
keyword_representation.head(5)

,2손다,3스픈,ᆢ,ᆢ저는,ㅋ,ㅋ넘,ㅋ존,ㅎ,ㅡ넘,가,...,흰우유,흰자,흰콩,흰후추,흰후추가루,히,히트,힌트,힐링,힘
recipeID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 카테고리 + 키워드

In [215]:
recipe_representation = pd.concat([category_representation, keyword_representation], axis=1).fillna(0)


In [216]:
recipe_representation = recipe_representation.drop_duplicates()
print(recipe_representation.shape)
print(recipe_representation.describe())

(871, 4037)
             간 건강        골다공증        노화방지    머슬업 다이어트          변비     스트레스 해소  \
count  871.000000  871.000000  871.000000  871.000000  871.000000  871.000000   
mean     0.137773    0.137773    0.133180    0.137773    0.130884    0.137773   
std      0.506813    0.506813    0.498908    0.506813    0.494893    0.506813   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
75%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
max      2.000000    2.000000    2.000000    2.000000    2.000000    2.000000   

             암 예방        위 건강        장 건강   저칼로리 다이어트  ...         흰우유  \
count  871.000000  871.000000  871.000000  871.000000  ...  871.000000   
mean     0.126292    0.137773    0.140069    0.137773  ...    0.005980   
std      0.486730    0.506813    0.5

In [217]:
def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    result_df = pd.DataFrame(data=cos_sim, index=[a.index])
    
    return result_df

In [218]:
cs_df = cos_sim_matrix(recipe_representation, recipe_representation)
cs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,861,862,863,864,865,866,867,868,869,870
recipeID,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.104553,0.099753,0.123909,0.115700,0.168121,0.093792,0.096656,0.212477,0.085763,...,0.000000,0.000173,0.000000,0.000000,0.000000,0.000106,0.000000,0.023696,0.014620,0.000000
1,0.104553,1.000000,0.081507,0.104183,0.097282,0.143414,0.079756,0.082452,0.181252,0.083657,...,0.000000,0.029175,0.000000,0.000000,0.000000,0.001682,0.002344,0.000000,0.001828,0.000000
2,0.099753,0.081507,1.000000,0.117808,0.204224,0.131063,0.072887,0.075367,0.165642,0.066859,...,0.000000,0.000009,0.000000,0.009905,0.000000,0.010524,0.000000,0.000000,0.000000,0.000050
3,0.123909,0.104183,0.117808,1.000000,0.115291,0.162801,0.090537,0.093617,0.205753,0.085460,...,0.000000,0.001542,0.000000,0.000037,0.000000,0.000938,0.002778,0.000000,0.002166,0.000030
4,0.115700,0.097282,0.204224,0.115291,1.000000,0.152015,0.084539,0.100553,0.194003,0.079900,...,0.000786,0.001976,0.012838,0.000000,0.000281,0.020494,0.038640,0.000000,0.002704,0.000029


In [219]:
cs_df.shape

(871, 871)

In [220]:
print(cs_df[0].sort_values(ascending=False)[1:4])

recipeID
8           0.212477
28          0.204769
35          0.203645
Name: 0, dtype: float64


In [221]:
print(data.loc[0])
print(data.loc[8])
print(data.loc[28])
print(data.loc[35])

recipeID                                                           0
recipe_name                                                 우엉양배추볶음밥
summary                                       찬밥이 기다려지는 우엉양배추볶음밥입니다.
ingredient_name    [재료] 찬 밥  적당량 양배추  적당량 우엉  적당량 느타리버섯  조금 다시마  ...
category                                                   저칼로리 다이어트
main_source        [찬, 밥, 적당량, 양배추, 적당량, 우엉, 적당량, 느타리버섯, 조금, 다시마,...
Name: 0, dtype: object
recipeID                                                           8
recipe_name                                                    곤약 무침
summary                                    다이어트에 좋다는 곤약무침을 만들어 보았네요.
ingredient_name    [재료] 곤약(250g)  1개 돌나물  100g 당근  10g 양파  10g [양...
category                                                   저칼로리 다이어트
main_source                                        [곤약, 돌나물, 당근, 양파]
Name: 8, dtype: object
recipeID                                                          28
recipe_name                                              

In [251]:
train_df, test_df = train_test_split(data_re, test_size=0.2, random_state=1234)

In [253]:
test_userids = list(set(test_df.user_id.values))

In [254]:
result_df = pd.DataFrame()

for user_id in tqdm(test_userids):
    user_record_df = train_df.loc[train_df.user_id == int(user_id), :]
    
    user_sim_df = cs_df.loc[user_record_df['recipeID']]
    user_rating_df = user_record_df[['rating']]

    sim_sum = np.sum(user_sim_df.T.to_numpy(), -1)     # 각 열의 값을 합산
    prediction = np.matmul(user_sim_df.T.to_numpy(), user_rating_df.to_numpy()).flatten() / (sim_sum+1)
    
    # 프린트 하는 과정
    prediction_df = pd.DataFrame(prediction, index=cs_df.index).reset_index()
    prediction_df.columns = ['recipeID', 'pred_rating']
    prediction_df = prediction_df[['recipeID', 'pred_rating']][prediction_df.recipeID.isin(test_df[test_df.user_id == user_id]['recipeID'])]
    
    temp_df = prediction_df.merge(test_df[test_df.user_id == user_id], on='recipeID')
    result_df = pd.concat([result_df, temp_df], axis=0)

100%|██████████| 695/695 [00:06<00:00, 106.77it/s]


In [255]:
result_df.head(2)

,recipeID,pred_rating,category,recipe_name,user_id,rating,review_content
0,824,0.0,치매예방,청국장찌개 황금레시피,1181,5,"아,맛있습니다."
0,458,0.0,피로회복,버섯 들깨탕 만드는 법,1415,5,국간장은 기호에 맞게 가감하세요\r\n전 반만 넣었는데 좀 짰어요\r\n그리고 찹쌀...


In [257]:
print(f"RMSE: {sqrt(mean_squared_error(result_df['rating'].values, result_df['pred_rating'].values))}")

RMSE: 4.869690033239285


# 협업필터링

## 아이템 기반

In [222]:
train_df, test_df = train_test_split(data_re, test_size=0.2, random_state=1234)

In [223]:
train_df.shape, test_df.shape

((2824, 6), (706, 6))

In [224]:
user_ids = sorted(set(train_df['user_id'].values))
recipe_ids = sorted(set(train_df['recipeID'].values))

print(f'유저 수: {len(user_ids)}, 레시피 수: {len(recipe_ids)}')


유저 수: 2645, 레시피 수: 351


In [225]:
sparse_matrix = pd.DataFrame(index=recipe_ids, columns=user_ids)
sparse_matrix

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [226]:
train_df.head(10)

,category,recipeID,recipe_name,user_id,rating,review_content
1348,스트레스 해소,376,불닭볶음면+틈새라면=지옥의 염라면 만들기,765,5,맛있네요!
3259,혈액순환,479,부추무침,1108,5,정말 맛있어요^^ 잘 이용하겠습니당
235,골다공증,197,통조림으로 만든 집밥 백선생 일본식 꽁치조림,62,5,매번 레시피 잊어버리는데 만 개의 레시피를 보고 다시 떠올리곤 합니다. ㅎ
671,머슬업 다이어트,84,고등어무조림,2087,5,레시피대로햇더니 맛나네요~~생강가루 조금 첨가했어요 울아찌 맛있다고 한그릇 뚝딱햇어요
89,간 건강,784,비오는날에는 ★새우부추전,2569,5,간단하게 할수 있고 영양 좋은 음식이라\r\n좋았읍니다.
2655,치매예방,869,절대 후회없는 깻잎김치 황금레시피^^,2083,5,색깔이며 맛이며 좋아요^^\r\n다만 깻잎두장씩 해서했는데도 양념이 계량된데로하니 ...
507,머슬업 다이어트,84,고등어무조림,1107,5,이대로만 하시면 엄마가 해주시던 고등어조림 맛이 날겁니다. \r\n저는 무가 푹 익...
29,간 건강,768,생생정보 오이미역냉국 황금레시피,733,5,미역오이냉국이예여\r\n함올려봣어여\r\n맛잇겟죠
218,골다공증,196,송일국 꽁치쌈장 만들기 ★,3219,5,강추! 너무 맛있어요. 상추쌈에도 굿~ 입니다. 감사해요.
1130,머슬업 다이어트,110,닭가슴살 갈비구이,3169,5,만들기 쉽고 맛있어요~ 저는 간장 4스푼 넣었구요. 닭만 먹으면 조금 짠데 밥이나 ...


In [227]:
train_df_grouped = train_df.groupby(['recipeID', 'user_id'])['rating'].mean().reset_index()
sparse_matrix = train_df_grouped.pivot(index='recipeID', columns='user_id', values='rating')
sparse_matrix.index.name = 'recipeID'
sparse_matrix

user_id,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
recipeID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [228]:
def cossim_matrix(a, b):
    cossim_values = cosine_similarity(a.values, b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns = a.index.values, index=a.index)

    return cossim_df

In [229]:
item_sparse_matrix = sparse_matrix.fillna(0)
item_sparse_matrix

user_id,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
recipeID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [230]:
item_cossim_df = cossim_matrix(item_sparse_matrix, item_sparse_matrix)
item_cossim_df

,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
recipeID,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.022421,0.000000,0.0
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0
869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.042741,0.022421,0.0,1.000000,0.052497,0.0


In [280]:
def get_item_based_collabor(title):
    return item_cossim_df[title].sort_values(ascending=False)[:5]

In [285]:
get_item_based_collabor(863)

recipeID
863    1.000000
869    0.042741
1      0.000000
604    0.000000
617    0.000000
Name: 863, dtype: float64

In [289]:
print(data.loc[data['recipeID'] == 863, 'recipe_name'])
print(data.loc[data['recipeID'] == 869, 'recipe_name'])

863    초간단한 조리법, 개운한 양파 겉절이
Name: recipe_name, dtype: object
869    절대 후회없는 깻잎김치 황금레시피^^
Name: recipe_name, dtype: object


In [231]:
userId_grouped = train_df.groupby('user_id')
# userId_grouped.indices.keys() : user_id
# item_sparse_matrix.index : recipe_id
item_prediction_result_df = pd.DataFrame(index=list(userId_grouped.indices.keys()), columns=item_sparse_matrix.index)
item_prediction_result_df

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [232]:
for user_id, group in tqdm(userId_grouped):
    user_sim = item_cossim_df.loc[group['recipeID']]   # 유저의 레시피 유사도
    user_rating = group['rating']   # 유저의 평점

    # 사용자가 평가한 레시피들의 유사도 총합을 계산
    sim_sum = user_sim.sum(axis=0)
    
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    item_prediction_result_df.loc[user_id] = pred_ratings

 17%|█▋        | 461/2645 [00:01<00:06, 341.90it/s]

100%|██████████| 2645/2645 [00:04<00:00, 615.30it/s]


In [233]:
item_prediction_result_df.head(10)

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.137207,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.231965,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.104691,0.0,0.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.144901,0.0,0.0,0.214043,0.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.227307,0.0,0.0,0.0,0.0
1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.204947,0.109648,0.0,2.5,0.249394,0.0


In [234]:
print(item_prediction_result_df.shape)

(2645, 351)


## 유저 기반

In [235]:
user_sparse_matrix = sparse_matrix.fillna(0).transpose()
user_sparse_matrix.head(5)

recipeID,1,2,4,8,9,11,12,13,14,16,...,851,852,856,862,863,865,868,869,870,872
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
user_sparse_matrix.shape

(2645, 351)

In [237]:
user_cossim_df = cossim_matrix(user_sparse_matrix, user_sparse_matrix)
user_cossim_df

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [238]:
recipeId_grouped = train_df.groupby('recipeID')
user_prediction_result_df = pd.DataFrame(index=list(recipeId_grouped.indices.keys()), columns=user_sparse_matrix.index)
user_prediction_result_df

user_id,0,1,10,100,1000,1001,1002,1003,1004,1005,...,981,983,984,985,986,988,990,991,992,997
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [239]:
for recipeID, group in tqdm(recipeId_grouped):
    user_sim = user_cossim_df.loc[group['user_id']]
    user_rating = group['rating']
    sim_sum = user_sim.sum(axis=0)
    
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    user_prediction_result_df.loc[recipeID] = pred_ratings

100%|██████████| 351/351 [00:01<00:00, 244.67it/s]


In [240]:
user_prediction_result_df = user_prediction_result_df.transpose()

In [241]:
print(user_prediction_result_df.shape)
print(item_prediction_result_df.shape)

(2645, 351)
(2645, 351)


In [242]:
test_df.head()

,category,recipeID,recipe_name,user_id,rating,review_content
2766,피로회복,441,문어 꽈리고추 조림,1421,3,문어가 없어서 오징어로 대체해봤어요
63,간 건강,779,새우 브로콜리 볶음,1296,5,새우가 냉동실에 많아 다 쏟아붓고했더니 ㅋㅋ 많은양에 양념을 못맞춰 이것더것 조금씩...
3525,호흡기 건강,586,겨울철 가족의 건강을 지켜주는 대추고(차) 만드는법,3177,5,진짜 열심히 만들었는데 만들고 보니 된장 같이 생겼네요 ㅋㅋ 물에 타먹으니 맛있어요 ^^
2126,장 건강,325,느타리버섯양념구이,596,5,맛있어요~ 벌써 여러 번 해 먹었답니다 ^^
25,간 건강,768,생생정보 오이미역냉국 황금레시피,52,5,너무맞있게먹었요.


In [268]:
# def evaluate(test_df, prediction_result_df):
#   groups_with_movie_ids = test_df.groupby(by='recipeID')
#   groups_with_user_ids = test_df.groupby(by='user_id')
  
#   # 트레인 테스트 df에도 있는 무비 ID와 유저 아이디 가지고 오기
#   intersection_movie_ids = sorted(list(set(list(prediction_result_df.columns)). \
#       intersection(set(list(groups_with_movie_ids.indices.keys())))))
#   intersection_user_ids = sorted(list(set(list(prediction_result_df.index)).intersection(set(groups_with_user_ids.indices.keys()))))

#   print(len(intersection_movie_ids))
#   print(len(intersection_user_ids))
  
#   compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_movie_ids]
  
#   grouped = test_df.groupby(by='user_id')
# #   result_df = pd.DataFrame(columns=['rmse'])
  
#   for user_id, group in tqdm(grouped):
#       if user_id in intersection_user_ids:
#           pred_ratings = compressed_prediction_df.loc[user_id][compressed_prediction_df.loc[user_id].index.\
#               intersection(list(group['recipeID'].values))]
#           pred_ratings = pred_ratings.to_frame(name='rating').reset_index().\
#               rename(columns={'index':'recipeID','rating':'pred_rating'})
#           actual_ratings = group[['rating', 'recipeID']].rename(columns={'rating':'actual_rating'})
          
#           final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['recipeID'])
#           final_df = final_df.round(4)
          
#   return final_df
  

In [274]:
def evaluate(test_df, prediction_result_df):
    groups_with_recipe_ids = test_df.groupby(by='recipeID')
    groups_with_user_ids = test_df.groupby(by='user_id')
  
    intersection_recipe_ids = sorted(list(set(list(prediction_result_df.columns)).intersection(set(list(groups_with_recipe_ids.indices.keys())))))
    intersection_user_ids = sorted(list(set(list(prediction_result_df.index)).intersection(set(groups_with_user_ids.indices.keys()))))

    print(len(intersection_recipe_ids))
    print(len(intersection_user_ids))
  
    compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_recipe_ids]
  
    grouped = test_df.groupby(by='user_id')
    result_df_list = []  # List to store final_df for each user
  
    for user_id, group in tqdm(grouped):
        if user_id in intersection_user_ids:
            pred_ratings = compressed_prediction_df.loc[user_id][compressed_prediction_df.loc[user_id].index.\
                intersection(list(group['recipeID'].values))]
            pred_ratings = pred_ratings.to_frame(name='rating').reset_index().\
                rename(columns={'index':'recipeID','rating':'pred_rating'})
            actual_ratings = group[['rating', 'recipeID']].rename(columns={'rating':'actual_rating'})
          
            final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['recipeID'])
            final_df = final_df.round(4)
            
            result_df_list.append(final_df)  # Append final_df to result_df_list

    # Concatenate all dataframes in result_df_list
    result_df = pd.concat(result_df_list, ignore_index=True)
    
    return result_df

In [275]:
evaluate(test_df, user_prediction_result_df)

171
69


  0%|          | 0/695 [00:00<?, ?it/s]

100%|██████████| 695/695 [00:00<00:00, 1750.16it/s]


,actual_rating,recipeID,pred_rating
0,5,73,0.0
1,5,84,3.328633
2,5,613,1.25
3,4,750,0.0
4,5,618,0.0
...,...,...,...
68,5,356,0.0
69,5,743,0.0
70,5,845,0.0
71,5,625,0.0


In [276]:
evaluate(test_df, item_prediction_result_df)

171
69


  0%|          | 0/695 [00:00<?, ?it/s]

100%|██████████| 695/695 [00:00<00:00, 1190.43it/s]


,actual_rating,recipeID,pred_rating
0,5,73,0.0
1,5,84,0.057446
2,5,613,0.081667
3,4,750,0.0
4,5,618,0.0
...,...,...,...
68,5,356,0.0
69,5,743,0.0
70,5,845,0.0
71,5,625,0.0


In [277]:
result_df = evaluate(test_df, user_prediction_result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

171
69


  0%|          | 0/695 [00:00<?, ?it/s]

100%|██████████| 695/695 [00:00<00:00, 1396.00it/s]

RMSE: 4.462581488251801


In [278]:
result_df = evaluate(test_df, item_prediction_result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

171
69


  0%|          | 0/695 [00:00<?, ?it/s]

100%|██████████| 695/695 [00:00<00:00, 1996.48it/s]

RMSE: 4.852610075163307
